In [3]:
import os
import pathlib
from datetime import datetime
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import pandas as pd

In [5]:
# set folders
raw = 'raw_data_srag_version-' + datetime.now().strftime('%Y-%m-%d')
output_srag = 'data-sindrome_respiratoria_aguda_grave_incluindo_covid/'

pathlib.Path(output_srag).mkdir(parents=True, exist_ok=True)

url = 'https://opendatasus.saude.gov.br/dataset/'
endpoints = [
    'srag-2009-2012',
    'srag-2013-2018',
    'srag-2019',
    'srag-2020',
    'srag-2021-e-2022',
]

for endpoint in endpoints:
    r = requests.get(os.path.join(url, endpoint))

    soup = BeautifulSoup(r.text, 'html.parser')

    tag_a = soup.findAll('a', class_='heading')

    links = []
    for tag in tag_a:
        string = tag.text.lstrip('\n').rstrip('\n').lstrip(' ').rstrip(' ')[0:4]
        if string == 'SRAG':
            links.append('/'.join(tag['href'].split('/')[2:]))


    for link in links:
        r = requests.get(os.path.join(url, link))

        soup = BeautifulSoup(r.text, 'html.parser')

        tag_a = soup.findAll('a', class_='resource-url-analytics')

        for tag in tag_a:
            if tag['href'].endswith('.csv') and tag['href'] == tag.text:
                file_csv = requests.get(tag['href'], stream=True)
                name = ''.join(tag.text.split('/')[-1:])

                with open(os.path.join(output_srag, name), 'wb') as f, tqdm(
                    desc=tag.text,
                    # total=int(file_csv.headers['Content-Length']),
                    unit='iB',
                    unit_scale=True,
                    unit_divisor=1024
                ) as bar:
                    for content in file_csv.iter_content(chunk_size=1024):
                        size = f.write(content)
                        bar.update(size)

https://d26692udehoye.cloudfront.net/SRAG/2009-2012/INFLUD09.csv: 20.7MiB [00:01, 15.5MiB/s]
https://d26692udehoye.cloudfront.net/SRAG/2009-2012/INFLUD10.csv: 2.71MiB [00:00, 8.48MiB/s]
https://d26692udehoye.cloudfront.net/SRAG/2009-2012/INFLUD11.csv: 1.09MiB [00:00, 5.01MiB/s]
https://d26692udehoye.cloudfront.net/SRAG/2009-2012/INFLUD12.csv: 5.53MiB [00:00, 17.1MiB/s]
https://d26692udehoye.cloudfront.net/SRAG/2013-2018/INFLUD13.csv: 10.6MiB [00:00, 11.4MiB/s]
https://d26692udehoye.cloudfront.net/SRAG/2013-2018/INFLUD14.csv: 5.56MiB [00:00, 14.6MiB/s]
https://d26692udehoye.cloudfront.net/SRAG/2013-2018/INFLUD15.csv: 4.25MiB [00:00, 12.1MiB/s]
https://d26692udehoye.cloudfront.net/SRAG/2013-2018/INFLUD16.csv: 15.9MiB [00:01, 8.98MiB/s]
https://d26692udehoye.cloudfront.net/SRAG/2013-2018/INFLUD17.csv: 8.76MiB [00:00, 20.6MiB/s]
https://d26692udehoye.cloudfront.net/SRAG/2013-2018/INFLUD18.csv: 14.0MiB [00:01, 12.0MiB/s]
https://d26692udehoye.cloudfront.net/SRAG/2019/INFLUD19.csv: 21.0MiB [